In [2]:
!pip install chromedriver_autoinstaller

In [1]:
!pip install pywin32

In [1]:
import re
from urllib.request import urlopen

import requests 
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import chromedriver_autoinstaller
import subprocess

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

import shutil
import time

In [2]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61'}

In [3]:
## 드라이버 키기.
subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument(headers)

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

# 스킨토너 크롤링

In [35]:
## 드라이버 키기.

subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument(headers)


chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

In [36]:
# 스킨케어 제품 끌어오기
brand = [] # 브랜드
name = [] # 제품명
bottle = [] # 용량
price_num = [] # 가격
star = [] # 제품 별점
product_info = [] # 제품 설명
product_type = [] # 제품 타입
ingredient = [] # 제품 성분
danger_num = [] # 위험성분 개수
rank_num = [] # 제품 순위

age = [] # 리뷰_나이
skintype = [] # 리뷰_스킨타입
gender = [] # 리뷰_성별
rating = [] # 리뷰_별점
review = []# 리뷰
offer = []# 리뷰_평가단 여부
offer_text = '평가단  제품을 무상으로 제공받아 사용 후 솔직하게 작성된 리뷰입니다.'
skin_tonur_image_link = []

for category in range(1,5):
    url = 'https://www.glowpick.com/categories/1?ids={}'.format(category)
    print(category)
    
    driver.get(url)
    time.sleep(2)
    if category == 1:
        driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div[2]/div/div[6]/button/span').click()
        driver.find_element_by_xpath(f'//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div[2]/button[1]').click()
    web_page = BeautifulSoup(driver.page_source, 'html.parser')
    link_len = len(web_page.select('li.ranking__list__item'))

    for i in range(link_len):
        #스크롤 이동
        time.sleep(1)
        action = ActionChains(driver)
        try:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+2}]')).perform()
        except:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]')).perform()
            time.sleep(1)
            pass
        time.sleep(1)
        #화장품선택
        button = driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]').click()
        driver.execute_script("arguments[0];", button)
        time.sleep(3)
        
        #성분 창 클릭
        driver.find_element_by_xpath('//*[@id="contents"]/section/div[3]/article[2]/h3/button').click()
        time.sleep(3)
            #성분 텍스트 가져오기
        try:
            ingredient_contents = driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div/ul').text
            time.sleep(1)
        except:
            #수상 내역 창 끄기
            driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/h1/button/span').click()
            time.sleep(2)
            #성분 창 클릭
            driver.find_element_by_xpath('//*[@id="contents"]/section/div[3]/article[3]/h3/button').click()
            time.sleep(2)
            ingredient_contents = driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div/ul').text
        
        #성분 창 끄기
        driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/h1/button/span').click()

        time.sleep(1)
        
        web_page = BeautifulSoup(driver.page_source, 'html.parser')
        #리뷰 항목 데이터 끌어오기
        detail_info = web_page.find_all('article', {"class":"review reviews__wrapper__review"})
        #상품 메인 데이터 끌어오기
        detail_info_2 = web_page.find_all('div',{'class':'product__summary'})
        #상품 서브 데이터 끌어오기
        sub_info = web_page.find('pre',{'class':'info__text description__text'})

        #상품 해시태그 자료 끌어오기
        if web_page.find('p',{'class':'description__list'}):
            p_type = web_page.find('p',{'class':'description__list'}).get_text().replace('\n','')
            p_type = p_type.lstrip().split('#')
            p_type = ' '.join(list(map(str.strip,p_type))).lstrip()
        else:
            p_type = '없음'
        
        ##위험치수 가져오기
        if web_page.find('span',{'class':'info__text info__text-red'}):
            danger = web_page.find('span',{'class':'info__text info__text-red'}).get_text()
            danger = danger.replace('\n','').replace(' ','').replace('개','')
        else:
            danger = '없음'

        # 제품 순위 가져오기
        rank = web_page.find('span',{'class':'rank-item__rank'}).get_text()


        for k in range(len(detail_info)):
            
            detail = detail_info[k].get_text().split('\n')

            detail = list(map(str.strip, detail))
            
            # 성분 끌어오기
            ingredient.append(ingredient_contents)
            # 제품 정보 끌어오기
            product_info.append(sub_info.get_text().replace('\n','').replace('\r',''))
            # 제품 타입 끌어오기
            product_type.append(p_type)
            # 위험성분 개수 끌어오기
            danger_num.append(danger)
            # 순위 끌어오기
            rank_num.append(rank.replace('위',''))
            #제품정보 끌어오기
            product_cataloge = detail_info_2[0].get_text().split('\n')
            #브랜드명 끌어오기
            brand.append(product_cataloge[1].lstrip())
            #제품명 끌어오기
            name.append(product_cataloge[2].lstrip())
            #용량 끌어오기
            bottle.append(product_cataloge[4].strip().split('/')[0])
            #가격 끌어오기
            price = product_cataloge[4].strip().split('/')[1]
            price = price.replace('원','')
            price_num.append(price)
            #제품의 별점 끌어오기
            star.append(product_cataloge[5].lstrip()[:4])            
            # 리뷰 나이 끌어오기
            age.append(detail[1][:2])
            # 리뷰_스킨타입
            skintype.append(detail[3])
            # 리뷰_성별
            gender.append(detail[5])
            # 리뷰_별점
            rating.append(detail[9])
            # 이미지 링크
            skin_tonur_image_link.append(web_page.text.split('Product')[1].split('"image":"')[1].split('","brand"')[0])
            # 리뷰
            review_l = ' '.join(detail[12:])[:-2]
            
            if offer_text in review_l:
                re = review_l.replace(offer_text,'')
                review.append(re.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(1)
            else:
                review.append(review_l.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(0)

        driver.back()
        time.sleep(1)

23
24
25
26


In [34]:
driver.close()
driver.quit()

In [37]:
print(len(brand))
print(len(name))
print(len(bottle))
print(len(price_num))
print(len(star))
print(len(product_info))
print(len(product_type))
print(len(ingredient))
print(len(danger_num))
print(len(rank_num))

print(len(age))
print(len(skintype))
print(len(gender))
print(len(rating))
print(len(review))
print(len(offer))
print(len(skin_tonur_image_link))

3010
3010
3010
3010
3010
3010
3010
3010
3010
3010
3010
3010
3010
3010
3010
3010
3010


In [38]:
skin_tonur = pd.DataFrame({
    'brand' : brand,
    'name' : name,
    'bottle': bottle,
    'price_num' : price_num,
    'star' : star,
    'product_info' : product_info,
    'product_type' : product_type,
    'ingredient' : ingredient,
    'danger_num' : danger_num,
    'rank_num' : rank_num,
    'age' : age,
    'skintype' : skintype,
    'gender' : gender,
    'rating' : rating,
    'review' : review,
    'offer' : offer,
    'skin_tonur_image_link': skin_tonur_image_link,
})

In [7]:
skin_tonur

,brand,name,bottle,price_num,star,product_info,product_type,ingredient,danger_num,rank_num,age,skintype,gender,rating,review,offer,esence_image_link
0,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,34,복합성,여,3,그냥저냥 무난한 제품. 딱히 촉촉한거 같지도 않은데 왜 그렇게 유명하고 인기가 많은...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...
1,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,26,건성,여,3,묽고 깔끔한 느낌이 들고 흡수도 잘됩니다. 소량만 짜서 발라도 얼굴이 시원해지는 느...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...
2,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,28,지성,여,4,이름은 에센스지만 토너와 로션 중간쯤 되는 제형이라 약간 애매하지 싶어요 요즘 계절...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...
3,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,17,민감성,여,5,흡수도 잘되고 피부진정도 잘되고 촉촉해서 너무 좋아요!!진짜 한통 다썻는데 또 사서...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...
4,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,27,복합성,여,5,이거 신제품일 때(작년?) 올영에서 매우 저렴하게 구매해서 지금도 잘 쓰는 중.. ...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...
5,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,27,건성,여,3,딱히 특이점이없어요 피부가 좋아지거나 촉촉해지는건 못느끼고 살짝 트러블이올라오는데 ...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...
6,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,30,복합성,여,4,실패하지는 않을 선택 유난스러운 트러블 피부인데 아무것도 더 안남(제일 중요) 겁...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...
7,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,29,복합성,여,5,순하고 촉촉한 에센스에요~~ 자극없고 너무 좋아요~ 에센스인데 보습도 나쁘지않아요...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...
8,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,29,건성,여,5,정말 기본템에 순한템 순한걸로는 믿고 쓰는 아토베리어 피부 뒤집어졌을때 자극주기싫을...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...
9,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.46,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",3,1,28,복합성,여,5,무난하게 쓰기좋은 순한 에센스 피부가 예민한 편이라서 기능성제품은 왠만하면 사용 안...,0,https://dn5hzapyfrpio.cloudfront.net/product/a...


In [39]:
import re

In [ ]:
#열별로 제거 필요한 항목 함수 지정.
# def rmEmoji(inputdata):
#     return inputdata.encode('euc-kr', 'ignore').decode('euc-kr')

def text_ingredient(ingredient):
    return re.sub("[\n\r0-9-]","",ingredient)

# def text_product_info(product_info):
#     return re.sub("[\n\r]","",product_info)

In [40]:
## 특수문자 무시하고 텍스트 데이터만 추출하여 저장
for i in range(len(skin_tonur['ingredient'])):
    skin_tonur['ingredient'][i] = text_ingredient(skin_tonur['ingredient'][i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
skin_tonur

,brand,name,bottle,price_num,star,product_info,product_type,ingredient,danger_num,rank_num,age,skintype,gender,rating,review,offer,crim_image_link
0,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,50,건성,여,4,영양감 많은 단백하고 묵직한 발림성 단백한 그리고 쫀쫀한 영양감 많은 아이크림 바르...,0,https://dn5hzapyfrpio.cloudfront.net/product/7...
1,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,28,지성,남,4,글픽 설문단으로 당첨되서 작성하는 후기입니다 소량으로만 발라도 쫀쫀함이 느껴져요 뭔...,0,https://dn5hzapyfrpio.cloudfront.net/product/7...
2,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,38,건성,여,5,요즘 쓰는 아이크림인디 무난해요 조금 무거운 제형이라 오래 흡수시켜줘야해요 좋엏,0,https://dn5hzapyfrpio.cloudfront.net/product/7...
3,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,37,중성,여,5,쫀쫀한발림성굿 아이크림은 노화를막아주는 필수템인데 쫀쫀한 발려지고 흡수력이좋았어요,0,https://dn5hzapyfrpio.cloudfront.net/product/7...
4,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,27,복합성,남,4,끈적이지 않고 가벼워서 마음에 쏘옥 들었어요!,0,https://dn5hzapyfrpio.cloudfront.net/product/7...
5,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,26,건성,여,5,아이크림 유목민인데 순위도 좋고 리뷰도 좋아서 구매해봤어요 아직 한통을 다 못써서 ...,0,https://dn5hzapyfrpio.cloudfront.net/product/7...
6,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,20,건성,여,4,그냥 진짜 무난함. 안 끅적거리고 가벼움. 20대한테 추천함.,0,https://dn5hzapyfrpio.cloudfront.net/product/7...
7,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,33,민감성,여,4,무난템~~ 눈가에바르고 입술에도 바르기괜찮았어요 끈적이지않고 촉촉하고 좋아요,0,https://dn5hzapyfrpio.cloudfront.net/product/7...
8,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,26,복합성,여,4,너무 많이 바르면 기름지고 수분감도 좋고 흡수도 잘 되는 편이에요,0,https://dn5hzapyfrpio.cloudfront.net/product/7...
9,퍼스트랩,프로바이오틱 아이크림,30ml,"55,000",4.55,어둠을 밝혀주는 프로바이오틱스 아이 크림- 페이스 겸용 멀티 아이크림- 탄력 있고 ...,기능성 멀티크림 미백 보습 영양공급 주름개선 피부탄력,"정제수Water;Aqua피부컨디셔닝제, 용제글리세린Glycerin변성제, 착향제, ...",7,1,50,지성,남,4,설문단 당첨* 산뜻한 아이크림이예요. 확실히 더 영양감있는 아이크림 보다는 좀 더 ...,0,https://dn5hzapyfrpio.cloudfront.net/product/7...


In [42]:
skin_tonur.to_excel('skin_tonur.xlsx', encoding='euc-kr', index = False)

# 로션/에멀젼

In [ ]:
## 드라이버 키기.

subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument(headers)


chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

In [ ]:
# 로션/에멀젼 제품 끌어오기
brand = [] # 브랜드
name = [] # 제품명
bottle = [] # 용량
price_num = [] # 가격
star = [] # 제품 별점
product_info = [] # 제품 설명
product_type = [] # 제품 타입
ingredient = [] # 제품 성분
danger_num = [] # 위험성분 개수
rank_num = [] # 제품 순위

age = [] # 리뷰_나이
skintype = [] # 리뷰_스킨타입
gender = [] # 리뷰_성별
rating = [] # 리뷰_별점
review = []# 리뷰
offer = []# 리뷰_평가단 여부
offer_text = '평가단  제품을 무상으로 제공받아 사용 후 솔직하게 작성된 리뷰입니다.'
lotion_image_link = []

for category in range(5,11):
    url = 'https://www.glowpick.com/categories/2?ids={}'.format(category)
    print(category)
    
    driver.get(url)
    time.sleep(2)
    if category == 5:
        driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div[2]/div/div[6]/button/span').click()
        driver.find_element_by_xpath(f'//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div[2]/button[1]').click()
    web_page = BeautifulSoup(driver.page_source, 'html.parser')
    link_len = len(web_page.select('li.ranking__list__item'))

    for i in range(link_len):
        #스크롤 이동
        time.sleep(1)
        action = ActionChains(driver)
        try:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+2}]')).perform()
        except:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]')).perform()
            time.sleep(1)
            pass
        time.sleep(1)
        #화장품선택
        button = driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]').click()
        driver.execute_script("arguments[0];", button)
        time.sleep(3)
        
        #성분 창 클릭
        driver.find_element_by_xpath('//*[@id="contents"]/section/div[3]/article[2]/h3/button').click()
        time.sleep(3)
            #성분 텍스트 가져오기
        try:
            ingredient_contents = driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div/ul').text
            time.sleep(1)
        except:
            #수상 내역 창 끄기
            driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/h1/button/span').click()
            time.sleep(2)
            #성분 창 클릭
            driver.find_element_by_xpath('//*[@id="contents"]/section/div[3]/article[3]/h3/button').click()
            time.sleep(2)
            ingredient_contents = driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div/ul').text
        
        #성분 창 끄기
        driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/h1/button/span').click()

        time.sleep(1)
        
        web_page = BeautifulSoup(driver.page_source, 'html.parser')
        #리뷰 항목 데이터 끌어오기
        detail_info = web_page.find_all('article', {"class":"review reviews__wrapper__review"})
        #상품 메인 데이터 끌어오기
        detail_info_2 = web_page.find_all('div',{'class':'product__summary'})
        #상품 서브 데이터 끌어오기
        sub_info = web_page.find('pre',{'class':'info__text description__text'})

        #상품 해시태그 자료 끌어오기
        if web_page.find('p',{'class':'description__list'}):
            p_type = web_page.find('p',{'class':'description__list'}).get_text().replace('\n','')
            p_type = p_type.lstrip().split('#')
            p_type = ' '.join(list(map(str.strip,p_type))).lstrip()
        else:
            p_type = '없음'
        
        ##위험치수 가져오기
        if web_page.find('span',{'class':'info__text info__text-red'}):
            danger = web_page.find('span',{'class':'info__text info__text-red'}).get_text()
            danger = danger.replace('\n','').replace(' ','').replace('개','')
        else:
            danger = '없음'

        # 제품 순위 가져오기
        rank = web_page.find('span',{'class':'rank-item__rank'}).get_text()


        for k in range(len(detail_info)):
            
            detail = detail_info[k].get_text().split('\n')

            detail = list(map(str.strip, detail))
            
            # 성분 끌어오기
            ingredient.append(ingredient_contents)
            # 제품 정보 끌어오기
            product_info.append(sub_info.get_text().replace('\n','').replace('\r',''))
            # 제품 타입 끌어오기
            product_type.append(p_type)
            # 위험성분 개수 끌어오기
            danger_num.append(danger)
            # 순위 끌어오기
            rank_num.append(rank.replace('위',''))
            #제품정보 끌어오기
            product_cataloge = detail_info_2[0].get_text().split('\n')
            #브랜드명 끌어오기
            brand.append(product_cataloge[1].lstrip())
            #제품명 끌어오기
            name.append(product_cataloge[2].lstrip())
            #용량 끌어오기
            bottle.append(product_cataloge[4].strip().split('/')[0])
            #가격 끌어오기
            price = product_cataloge[4].strip().split('/')[1]
            price = price.replace('원','')
            price_num.append(price)
            #제품의 별점 끌어오기
            star.append(product_cataloge[5].lstrip()[:4])            
            # 리뷰 나이 끌어오기
            age.append(detail[1][:2])
            # 리뷰_스킨타입
            skintype.append(detail[3])
            # 리뷰_성별
            gender.append(detail[5])
            # 리뷰_별점
            rating.append(detail[9])
            # 이미지 링크
            lotion_image_link.append(web_page.text.split('Product')[1].split('"image":"')[1].split('","brand"')[0])
            # 리뷰
            review_l = ' '.join(detail[12:])[:-2]
            
            if offer_text in review_l:
                re = review_l.replace(offer_text,'')
                review.append(re.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(1)
            else:
                review.append(review_l.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(0)

        driver.back()
        time.sleep(1)

In [ ]:
driver.close()
driver.quit()

In [ ]:
print(len(brand))
print(len(name))
print(len(bottle))
print(len(price_num))
print(len(star))
print(len(product_info))
print(len(product_type))
print(len(ingredient))
print(len(danger_num))
print(len(rank_num))

print(len(age))
print(len(skintype))
print(len(gender))
print(len(rating))
print(len(review))
print(len(offer))
print(len(lotion_image_link))

In [ ]:
lotion = pd.DataFrame({
    'brand' : brand,
    'name' : name,
    'bottle': bottle,
    'price_num' : price_num,
    'star' : star,
    'product_info' : product_info,
    'product_type' : product_type,
    'ingredient' : ingredient,
    'danger_num' : danger_num,
    'rank_num' : rank_num,
    'age' : age,
    'skintype' : skintype,
    'gender' : gender,
    'rating' : rating,
    'review' : review,
    'offer' : offer,
    'lotion_image_link': lotion_image_link,
})

In [ ]:
lotion

In [ ]:
import re

In [ ]:
## 특수문자 무시하고 텍스트 데이터만 추출하여 저장
for i in range(len(lotion['ingredient'])):
    lotion['ingredient'][i] = text_ingredient(lotion['ingredient'][i])

In [ ]:
lotion

In [ ]:
lotion.to_excel('skin_tonur.xlsx', encoding='euc-kr', index = False)

# 에센스/세럼 

In [92]:
## 드라이버 키기.

subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument(headers)

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

In [93]:
# 에센스/세럼 제품 끌어오기
brand = [] # 브랜드
name = [] # 제품명
bottle = [] # 용량
price_num = [] # 가격
star = [] # 제품 별점
product_info = [] # 제품 설명
product_type = [] # 제품 타입
ingredient = [] # 제품 성분
danger_num = [] # 위험성분 개수
rank_num = [] # 제품 순위

age = [] # 리뷰_나이
skintype = [] # 리뷰_스킨타입
gender = [] # 리뷰_성별
rating = [] # 리뷰_별점
review = []# 리뷰
offer = []# 리뷰_평가단 여부
offer_text = '평가단  제품을 무상으로 제공받아 사용 후 솔직하게 작성된 리뷰입니다.'
esence_image_link = []


for category in range(11,18):
    url = 'https://www.glowpick.com/categories/3?ids={}'.format(category)
    print(category)
    
    driver.get(url)
    time.sleep(2)
    if category == 11:
        driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div[2]/div/div[6]/button/span').click()
        driver.find_element_by_xpath(f'//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div[2]/button[1]').click()
    web_page = BeautifulSoup(driver.page_source, 'html.parser')
    link_len = len(web_page.select('li.ranking__list__item'))

    for i in range(link_len):
        #스크롤 이동
        time.sleep(1)
        action = ActionChains(driver)
        try:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+2}]')).perform()
        except:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]')).perform()
            time.sleep(1)
            pass
        time.sleep(1)
        #화장품선택
        button = driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]').click()
        driver.execute_script("arguments[0];", button)
        time.sleep(3)
        
        #성분 창 클릭
        driver.find_element_by_xpath('//*[@id="contents"]/section/div[3]/article[2]/h3/button').click()
        time.sleep(3)
        #성분 텍스트 가져오기
        try:
            ingredient_contents = driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div/ul').text
            time.sleep(1)
        except:
            #수상 내역 창 끄기
            driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/h1/button/span').click()
            time.sleep(2)
            #성분 창 클릭
            driver.find_element_by_xpath('//*[@id="contents"]/section/div[3]/article[3]/h3/button').click()
            time.sleep(2)
            ingredient_contents = driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div/ul').text
        
        #성분 창 끄기
        driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/h1/button/span').click()

        time.sleep(1)
        
        web_page = BeautifulSoup(driver.page_source, 'html.parser')
        #리뷰 항목 데이터 끌어오기
        detail_info = web_page.find_all('article', {"class":"review reviews__wrapper__review"})
        #상품 메인 데이터 끌어오기
        detail_info_2 = web_page.find_all('div',{'class':'product__summary'})
        #상품 서브 데이터 끌어오기
        sub_info = web_page.find('pre',{'class':'info__text description__text'})

        #상품 해시태그 자료 끌어오기
        if web_page.find('p',{'class':'description__list'}):
            p_type = web_page.find('p',{'class':'description__list'}).get_text().replace('\n','')
            p_type = p_type.lstrip().split('#')
            p_type = ' '.join(list(map(str.strip,p_type))).lstrip()
        else:
            p_type = '없음'
        
        ##위험치수 가져오기
        if web_page.find('span',{'class':'info__text info__text-red'}):
            danger = web_page.find('span',{'class':'info__text info__text-red'}).get_text()
            danger = danger.replace('\n','').replace(' ','').replace('개','')
        else:
            danger = '없음'

        # 제품 순위 가져오기
        rank = web_page.find('span',{'class':'rank-item__rank'}).get_text()


        for k in range(len(detail_info)):
            
            detail = detail_info[k].get_text().split('\n')

            detail = list(map(str.strip, detail))
            
            # 성분 끌어오기
            ingredient.append(ingredient_contents)
            # 제품 정보 끌어오기
            product_info.append(sub_info.get_text().replace('\n','').replace('\r',''))
            # 제품 타입 끌어오기
            product_type.append(p_type)
            # 위험성분 개수 끌어오기
            danger_num.append(danger)
            # 순위 끌어오기
            rank_num.append(rank.replace('위',''))
            #제품정보 끌어오기
            product_cataloge = detail_info_2[0].get_text().split('\n')
            #브랜드명 끌어오기
            brand.append(product_cataloge[1].lstrip())
            #제품명 끌어오기
            name.append(product_cataloge[2].lstrip())
            #용량 끌어오기
            bottle.append(product_cataloge[4].strip().split('/')[0])
            #가격 끌어오기
            price = product_cataloge[4].strip().split('/')[1]
            price = price.replace('원','')
            price_num.append(price)
            #제품의 별점 끌어오기
            star.append(product_cataloge[5].lstrip()[:4])            
            # 리뷰 나이 끌어오기
            age.append(detail[1][:2])
            # 리뷰_스킨타입
            skintype.append(detail[3])
            # 리뷰_성별
            gender.append(detail[5])
            # 리뷰_별점
            rating.append(detail[9])
            # 이미지 링크
            esence_image_link.append(web_page.text.split('Product')[1].split('"image":"')[1].split('","brand"')[0])
            # 리뷰
            review_l = ' '.join(detail[12:])[:-2]
            
            if offer_text in review_l:
                re = review_l.replace(offer_text,'')
                review.append(re.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(1)
            else:
                review.append(review_l.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(0)

        driver.back()
        time.sleep(1)

5
6
7
8
9
10


In [ ]:
driver.close()
driver.quit()

In [94]:
print(len(brand))
print(len(name))
print(len(bottle))
print(len(price_num))
print(len(star))
print(len(product_info))
print(len(product_type))
print(len(ingredient))
print(len(danger_num))
print(len(rank_num))

print(len(age))
print(len(skintype))
print(len(gender))
print(len(rating))
print(len(review))
print(len(offer))
print(len(esence_image_link))

3554
3554
3554
3554
3554
3554
3554
3554
3554
3554
3554
3554
3554
3554
3554
3554
3554


In [95]:
esence = pd.DataFrame({
    'brand' : brand,
    'name' : name,
    'bottle': bottle,
    'price_num' : price_num,
    'star' : star,
    'product_info' : product_info,
    'product_type' : product_type,
    'ingredient' : ingredient,
    'danger_num' : danger_num,
    'rank_num' : rank_num,
    'age' : age,
    'skintype' : skintype,
    'gender' : gender,
    'rating' : rating,
    'review' : review,
    'offer' : offer,
    'esence_image_link' : esence_image_link,
})

In [77]:
esence

,brand,name,bottle,price_num,star,product_info,product_type,ingredient,danger_num,rank_num,age,skintype,gender,rating,review,offer,skin_tonur_image_link
0,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,35,건성,남,4,콧물제형에 가까우며 흡수가 생각보다 빨라요 7스킨 7스킨하는데 이건 2스킨만해도 될...,0,https://dn5hzapyfrpio.cloudfront.net/product/5...
1,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,15,건성,여,5,저만 좋다고 생각한 게 아니라 많은 분들이 좋다고 하신 제품이네요..!! 깔끔하고 ...,0,https://dn5hzapyfrpio.cloudfront.net/product/5...
2,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,23,복합성,여,5,용량도 많고 끈적임이 없어서 너무 좋아요,0,https://dn5hzapyfrpio.cloudfront.net/product/5...
3,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,21,복합성,여,5,조금 미끈거리지만 얼굴에 바르고나면 쫀쫀해요! 이벤트로 받았던 것 같은데 용량도 많...,0,https://dn5hzapyfrpio.cloudfront.net/product/5...
4,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,31,복합성,여,4,친구가 있어서 써봤는데요 사용감이 쫀쫀한게 건성인피부에 수분보호막을 형성해주는것 같...,0,https://dn5hzapyfrpio.cloudfront.net/product/5...
5,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,16,민감성,여,5,자극적이지 않은 제품 얼굴이 되게 쫀쫀해지는거 같고 자극적이지 않아서 좋았어요,0,https://dn5hzapyfrpio.cloudfront.net/product/5...
6,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,39,복합성,여,4,좋습니다 미끌거리지만. 되직한게 오히려 수분을 잘잡네요,0,https://dn5hzapyfrpio.cloudfront.net/product/5...
7,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,27,복합성,여,5,살짝 미끌거리는 제형이라서 수분도 묵직하게 채워주는 느낌 이고 용기도 위생적이고 편...,0,https://dn5hzapyfrpio.cloudfront.net/product/5...
8,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,19,지성,여,4,물토너여서 쓰기가 너무 편하고 사람들 말처럼 무난해서 넘 좋은거 같아요~~,0,https://dn5hzapyfrpio.cloudfront.net/product/5...
9,주미소,워터풀 히알루론산 토너,250ml,"20,000",4.65,피부 갈증의 해답 히알루론산 수분길토너- 저분자부터 고분자까지 탄탄하게 채워주는 강...,보습 수분공급 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,19,지성,여,5,흐르는 콧물 제형으로 완전히 흡수된 후에는 피부가 쫀쫀해짐. 흡수는 빠른 편이고 제...,0,https://dn5hzapyfrpio.cloudfront.net/product/5...


In [96]:
import re
## 특수문자 무시하고 텍스트 데이터만 추출하여 저장
for i in range(len(esence['ingredient'])):
    esence['ingredient'][i] = text_ingredient(esence['ingredient'][i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [36]:
esence

,brand,name,bottle,price_num,star,product_info,product_type,ingredient,danger_num,rank_num,age,skintype,gender,rating,review,offer
0,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,29,복합성,여,5,순하고 촉촉한 에센스에요~~ 자극없고 너무 좋아요~ 에센스인데 보습도 나쁘지않아요...,0
1,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,29,건성,여,5,정말 기본템에 순한템 순한걸로는 믿고 쓰는 아토베리어 피부 뒤집어졌을때 자극주기싫을...,0
2,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,28,복합성,여,5,무난하게 쓰기좋은 순한 에센스 피부가 예민한 편이라서 기능성제품은 왠만하면 사용 안...,0
3,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,31,복합성,여,5,인생템... 너무잘맞아 가볍지만 건조한게 없어요.. 크림이랑 찰떡.. 겨울에도 가능...,0
4,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,27,민감성,여,4,여름 에센스 여름에 사용하기 좋은 에센스에요 로션 제형도 아니고 물 같은 제형도 아...,0
5,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,28,복합성,여,5,양도 많도 무난하게 쓰기 좋은 수분에센스! 보습감이 좋은데 흡수도 빨리 되고 자극도...,0
6,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,25,복합성,여,4,아벤느 에센스인로션이랑 비슷한 느낌으로 속보습을 채워주는 제품같은데 에센스인로션 보...,0
7,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,30,복합성,여,3,무난 너무 쏘쏘 그자체… 그냥 수분에센스! 무난무난 그 자체다.. 제형은 토너에 가...,0
8,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,28,지성,여,5,에센스 라기보단 약간 로션같은 느낌의 사용감이긴하지만 수부지인 저에겐 과하게 유수분...,0
9,에스트라,아토베리어 365 하이드로 에센스,150ml,"25,000",4.47,30가지 수분영양소를 채워주는 피부장벽 속보습 에센스- 피부 건조 및 속당김 해결-...,보습 수분공급 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",3,1,27,복합성,여,5,속건조는 수분에센스로! 속건조 잡아주는 에센스! 건조하다고 무조건 리치한 제품을 바...,0


In [97]:
esence.to_excel('esence.xlsx', encoding='euc-kr', index = False)

# 크림

In [63]:
## 드라이버 키기.

subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument(headers)


chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

In [64]:
# 크림 제품 끌어오기
brand = [] # 브랜드
name = [] # 제품명
bottle = [] # 용량
price_num = [] # 가격
star = [] # 제품 별점
product_info = [] # 제품 설명
product_type = [] # 제품 타입
ingredient = [] # 제품 성분
danger_num = [] # 위험성분 개수
rank_num = [] # 제품 순위

age = [] # 리뷰_나이
skintype = [] # 리뷰_스킨타입
gender = [] # 리뷰_성별
rating = [] # 리뷰_별점
review = []# 리뷰
offer = []# 리뷰_평가단 여부
offer_text = '평가단  제품을 무상으로 제공받아 사용 후 솔직하게 작성된 리뷰입니다.'
crim_image_link = []

for category in range(18,27):
    url = 'https://www.glowpick.com/categories/4?ids={}'.format(category)
    print(category)
    
    driver.get(url)
    time.sleep(2)
    if category == 18:
        driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div[2]/div/div[6]/button/span').click()
        driver.find_element_by_xpath(f'//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div[2]/button[1]').click()
    web_page = BeautifulSoup(driver.page_source, 'html.parser')
    link_len = len(web_page.select('li.ranking__list__item'))

    for i in range(link_len):
        #스크롤 이동
        time.sleep(1)
        action = ActionChains(driver)
        try:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+2}]')).perform()
        except:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]')).perform()
            time.sleep(1)
            pass
        time.sleep(1)
        #화장품선택
        button = driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]').click()
        driver.execute_script("arguments[0];", button)
        time.sleep(3)
        
        
        web_page = BeautifulSoup(driver.page_source, 'html.parser')
        #리뷰 항목 데이터 끌어오기
        detail_info = web_page.find_all('article', {"class":"review reviews__wrapper__review"})
        #상품 메인 데이터 끌어오기
        detail_info_2 = web_page.find_all('div',{'class':'product__summary'})
        #상품 서브 데이터 끌어오기
        sub_info = web_page.find('pre',{'class':'info__text description__text'})

        #상품 해시태그 자료 끌어오기
        if web_page.find('p',{'class':'description__list'}):
            p_type = web_page.find('p',{'class':'description__list'}).get_text().replace('\n','')
            p_type = p_type.lstrip().split('#')
            p_type = ' '.join(list(map(str.strip,p_type))).lstrip()
        else:
            p_type = '없음'
        

        # 제품 순위 가져오기
        rank = web_page.find('span',{'class':'rank-item__rank'}).get_text()


        for k in range(len(detail_info)):
            
            detail = detail_info[k].get_text().split('\n')

            detail = list(map(str.strip, detail))
            
            #브랜드명 끌어오기
            brand.append(product_cataloge[1].lstrip())
            #제품명 끌어오기
            name.append(product_cataloge[2].lstrip())
            #용량 끌어오기
            bottle.append(product_cataloge[4].strip().split('/')[0])
            #가격 끌어오기
            price = product_cataloge[4].strip().split('/')[1]
            price = price.replace('원','')
            price_num.append(price)
            
            # 리뷰_별점
            rating.append(detail[9])
            # 이미지 끌어오기
            crim_image_link.append(web_page.text.split('Product')[1].split('"image":"')[1].split('","brand"')[0])
            # 리뷰
            review_l = ' '.join(detail[12:])[:-2]
            
            if offer_text in review_l:
                re = review_l.replace(offer_text,'')
                review.append(re.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(1)
            else:
                review.append(review_l.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(0)

        driver.back()
        time.sleep(1)

27
28
29


In [ ]:
driver.close()
driver.quit()

In [65]:
print(len(brand))
print(len(name))
print(len(bottle))
print(len(price_num))
print(len(star))
print(len(product_info))
print(len(product_type))
print(len(ingredient))
print(len(danger_num))
print(len(rank_num))

print(len(age))
print(len(skintype))
print(len(gender))
print(len(rating))
print(len(review))
print(len(offer))
print(len(mist_image_link))

2163
2163
2163
2163
0
0
0
0
0
0
0
0
0
2163
2163
2163
2163


In [67]:
crim = pd.DataFrame({
    'brand' : brand,
    'name' : name,
    'bottle': bottle,
    'price_num' : price_num,
    'rating' : rating,
    'review' : review,
    'offer' : offer,
    'crim_image_link' : crim_image_link,
})

In [17]:
crim

,brand,name,bottle,price_num,star,product_info,product_type,ingredient,danger_num,rank_num,age,skintype,gender,rating,review,offer
0,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,27,민감성,여,4,밤에는 그닥.. 낮에는 좋음! 겨울철 아침에 바르려고 샀어요! 촉촉하고 수분감 있고...,0
1,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,40,복합성,여,4,우와 이름 그대로 워터풀 히알루론산 크림 대박!!! 촉촉하니 제 피부에 잘 맞아요 ...,0
2,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,25,복합성,여,5,"크림이 진짜 엄청 쫀쫀해요,,,! 자기전에 바르고 자면 아침에 피부 속까지 촉촉하고...",0
3,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,37,건성,여,5,수부지에 수분 길을 열어주는 수분크림 탄탄하게 채워지는 보습인자로 피부 수분 장벽을...,0
4,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,34,건성,여,4,자극없는 수분크림 수분크림 제형이 바라면 코팅? 생기는 느낌이라 저는 저녁에 사용 ...,0
5,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,22,민감성,여,4,향이 강하지 않고 촉촉해요 ~ 피부가 민감하기도 하고 향이 강한 거 정말 싫어하는데...,0
6,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,21,건성,여,5,건성+민감성에게 적극 추천 1.지금까지 썼던 크림중 최고좋음ㅠㅠㅠ 크림이 정말 쫀...,0
7,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,23,민감성,여,5,건성한테는 가벼울지도 수분크림 유목민이라 한제품 다 쓰면 다른제품 사서 쓰는데 이게...,0
8,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,35,건성,여,5,샘플링당첨! 스킨이랑 같이 쓰는 중입니다 수분 가득하고 쫀쫀해요 근데 각자쓰면 ...,0
9,주미소,워터풀 히알루론산 크림,50g,",00",4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"1\n정제수\nWater;Aqua\n피부컨디셔닝제, 용제\n1\n부틸렌글라이콜\nB...",0,1,31,민감성,여,5,샘플링 당첨되서 쓰고 있는데 요거 물건이네요 제형만 보면 되게 가벼운 젤타입인데 막...,0


In [47]:
import re
## 특수문자 무시하고 텍스트 데이터만 추출하여 저장
for i in range(len(crim['ingredient'])):
    crim['ingredient'][i] = text_ingredient(crim['ingredient'][i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [68]:
crim.to_excel('crim.xlsx', encoding='euc-kr', index = False)

# 미스트

In [47]:
## 드라이버 키기.

subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument(headers)


chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

In [48]:
# 미스트 끌어오기
brand = [] # 브랜드
name = [] # 제품명
bottle = [] # 용량
price_num = [] # 가격
star = [] # 제품 별점
product_info = [] # 제품 설명
product_type = [] # 제품 타입
ingredient = [] # 제품 성분
danger_num = [] # 위험성분 개수
rank_num = [] # 제품 순위

age = [] # 리뷰_나이
skintype = [] # 리뷰_스킨타입
gender = [] # 리뷰_성별
rating = [] # 리뷰_별점
review = []# 리뷰
offer = []# 리뷰_평가단 여부
offer_text = '평가단  제품을 무상으로 제공받아 사용 후 솔직하게 작성된 리뷰입니다.'
mist_image_link = []

for category in range(27,30):
    url = 'https://www.glowpick.com/categories/5?ids={}'.format(category)
    print(category)
    
    driver.get(url)
    time.sleep(2)
    if category == 27:
        driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div[2]/div/div[6]/button/span').click()
        driver.find_element_by_xpath(f'//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div[2]/button[1]').click()
    web_page = BeautifulSoup(driver.page_source, 'html.parser')
    link_len = len(web_page.select('li.ranking__list__item'))

    for i in range(link_len):
        #스크롤 이동
        time.sleep(1)
        action = ActionChains(driver)
        try:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+2}]')).perform()
        except:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]')).perform()
            time.sleep(1)
            pass
        time.sleep(1)
        #화장품선택
        button = driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]').click()
        driver.execute_script("arguments[0];", button)
        time.sleep(3)
        
        
        web_page = BeautifulSoup(driver.page_source, 'html.parser')
        #리뷰 항목 데이터 끌어오기
        detail_info = web_page.find_all('article', {"class":"review reviews__wrapper__review"})
        #상품 메인 데이터 끌어오기
        detail_info_2 = web_page.find_all('div',{'class':'product__summary'})
        #상품 서브 데이터 끌어오기
        sub_info = web_page.find('pre',{'class':'info__text description__text'})

        #상품 해시태그 자료 끌어오기
        if web_page.find('p',{'class':'description__list'}):
            p_type = web_page.find('p',{'class':'description__list'}).get_text().replace('\n','')
            p_type = p_type.lstrip().split('#')
            p_type = ' '.join(list(map(str.strip,p_type))).lstrip()
        else:
            p_type = '없음'
        

        # 제품 순위 가져오기
        rank = web_page.find('span',{'class':'rank-item__rank'}).get_text()


        for k in range(len(detail_info)):
            
            detail = detail_info[k].get_text().split('\n')

            detail = list(map(str.strip, detail))
            
            #브랜드명 끌어오기
            brand.append(product_cataloge[1].lstrip())
            #제품명 끌어오기
            name.append(product_cataloge[2].lstrip())
            #용량 끌어오기
            bottle.append(product_cataloge[4].strip().split('/')[0])
            #가격 끌어오기
            price = product_cataloge[4].strip().split('/')[1]
            price = price.replace('원','')
            price_num.append(price)
            
            # 리뷰_별점
            rating.append(detail[9])
            # 이미지 끌어오기
            mist_image_link.append(web_page.text.split('Product')[1].split('"image":"')[1].split('","brand"')[0])
            # 리뷰
            review_l = ' '.join(detail[12:])[:-2]
            
            if offer_text in review_l:
                re = review_l.replace(offer_text,'')
                review.append(re.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(1)
            else:
                review.append(review_l.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(0)

        driver.back()
        time.sleep(1)

6


In [ ]:
driver.close()
driver.quit()

In [49]:
print(len(brand))
print(len(name))
print(len(bottle))
print(len(price_num))
print(len(star))
print(len(product_info))
print(len(product_type))
print(len(ingredient))
print(len(danger_num))
print(len(rank_num))

print(len(age))
print(len(skintype))
print(len(gender))
print(len(rating))
print(len(review))
print(len(offer))
print(len(mist_image_link))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [50]:
mist = pd.DataFrame({
    'brand' : brand,
    'name' : name,
    'bottle': bottle,
    'price_num' : price_num,
    'star' : star,
    'product_info' : product_info,
    'product_type' : product_type,
    'ingredient' : ingredient,
    'danger_num' : danger_num,
    'rank_num' : rank_num,
    'age' : age,
    'skintype' : skintype,
    'gender' : gender,
    'rating' : rating,
    'review' : review,
    'offer' : offer,
    'mist_image_link' : mist_image_link,
})

In [ ]:
mist

In [51]:
import re
## 특수문자 무시하고 텍스트 데이터만 추출하여 저장
for i in range(len(mist['ingredient'])):
    mist['ingredient'][i] = text_ingredient(mist['ingredient'][i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [52]:
mist.to_excel('mist.xlsx', encoding='euc-kr', index = False)

# 페이스오일

In [ ]:
## 드라이버 키기.

subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument(headers)


chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

In [ ]:
# 페이스오일 끌어오기
brand = [] # 브랜드
name = [] # 제품명
bottle = [] # 용량
price_num = [] # 가격
star = [] # 제품 별점
product_info = [] # 제품 설명
product_type = [] # 제품 타입
ingredient = [] # 제품 성분
danger_num = [] # 위험성분 개수
rank_num = [] # 제품 순위

age = [] # 리뷰_나이
skintype = [] # 리뷰_스킨타입
gender = [] # 리뷰_성별
rating = [] # 리뷰_별점
review = []# 리뷰
offer = []# 리뷰_평가단 여부
offer_text = '평가단  제품을 무상으로 제공받아 사용 후 솔직하게 작성된 리뷰입니다.'
face_oil_image_link = []

fixed = [6]

for category in fixed:
    url = 'https://www.glowpick.com/categories/{}'.format(category)
    print(category)
    
    driver.get(url)
    time.sleep(2)
    if category == 6:
        driver.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div[2]/div/div[6]/button/span').click()
        driver.find_element_by_xpath(f'//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div[2]/button[1]').click()
    web_page = BeautifulSoup(driver.page_source, 'html.parser')
    link_len = len(web_page.select('li.ranking__list__item'))

    for i in range(link_len):
        #스크롤 이동
        time.sleep(1)
        action = ActionChains(driver)
        try:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+2}]')).perform()
        except:
            action.move_to_element(driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]')).perform()
            time.sleep(1)
            pass
        time.sleep(1)
        #화장품선택
        button = driver.find_element_by_xpath(f'//*[@id="contents"]/div/div/div[1]/div/ul/li[{i+1}]').click()
        driver.execute_script("arguments[0];", button)
        time.sleep(3)
        
        
        web_page = BeautifulSoup(driver.page_source, 'html.parser')
        #리뷰 항목 데이터 끌어오기
        detail_info = web_page.find_all('article', {"class":"review reviews__wrapper__review"})
        #상품 메인 데이터 끌어오기
        detail_info_2 = web_page.find_all('div',{'class':'product__summary'})
        #상품 서브 데이터 끌어오기
        sub_info = web_page.find('pre',{'class':'info__text description__text'})

        #상품 해시태그 자료 끌어오기
        if web_page.find('p',{'class':'description__list'}):
            p_type = web_page.find('p',{'class':'description__list'}).get_text().replace('\n','')
            p_type = p_type.lstrip().split('#')
            p_type = ' '.join(list(map(str.strip,p_type))).lstrip()
        else:
            p_type = '없음'
        

        # 제품 순위 가져오기
        rank = web_page.find('span',{'class':'rank-item__rank'}).get_text()


        for k in range(len(detail_info)):
            
            detail = detail_info[k].get_text().split('\n')

            detail = list(map(str.strip, detail))
            
            #브랜드명 끌어오기
            brand.append(product_cataloge[1].lstrip())
            #제품명 끌어오기
            name.append(product_cataloge[2].lstrip())
            #용량 끌어오기
            bottle.append(product_cataloge[4].strip().split('/')[0])
            #가격 끌어오기
            price = product_cataloge[4].strip().split('/')[1]
            price = price.replace('원','')
            price_num.append(price)
            
            # 리뷰_별점
            rating.append(detail[9])
            # 이미지 끌어오기
            face_oil_image_link.append(web_page.text.split('Product')[1].split('"image":"')[1].split('","brand"')[0])
            # 리뷰
            review_l = ' '.join(detail[12:])[:-2]
            
            if offer_text in review_l:
                re = review_l.replace(offer_text,'')
                review.append(re.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(1)
            else:
                review.append(review_l.encode('euc-kr', 'ignore').decode('euc-kr')) # 이모티콘 제거
                offer.append(0)

        driver.back()
        time.sleep(1)

In [ ]:
driver.close()
driver.quit()

In [ ]:
print(len(brand))
print(len(name))
print(len(bottle))
print(len(price_num))
print(len(star))
print(len(product_info))
print(len(product_type))
print(len(ingredient))
print(len(danger_num))
print(len(rank_num))

print(len(age))
print(len(skintype))
print(len(gender))
print(len(rating))
print(len(review))
print(len(offer))
print(len(face_oil_image_link))

In [ ]:
face_oil = pd.DataFrame({
    'brand' : brand,
    'name' : name,
    'bottle': bottle,
    'price_num' : price_num,
    'star' : star,
    'product_info' : product_info,
    'product_type' : product_type,
    'ingredient' : ingredient,
    'danger_num' : danger_num,
    'rank_num' : rank_num,
    'age' : age,
    'skintype' : skintype,
    'gender' : gender,
    'rating' : rating,
    'review' : review,
    'offer' : offer,
    'face_oil_image_link' : face_oil_image_link,
})

In [ ]:
face_oil

In [ ]:
import re
## 특수문자 무시하고 텍스트 데이터만 추출하여 저장
for i in range(len(mist['ingredient'])):
    mist['ingredient'][i] = text_ingredient(mist['ingredient'][i])

In [ ]:
face_oil

In [ ]:
face_oil.to_excel('face_oil.xlsx', encoding='euc-kr', index = False)

# 소분류 , 대분류 열 추가

In [44]:
import win32com.client

In [99]:
# 파이썬으로 엑셀파일 제어
import win32com.client

excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = True

# 제어할 엑셀파일 경로따라 가서 찾기
wb = excel.Workbooks.Open('C:\\Users\\User\\python 수업\\파이널\\skin_tonur.xlsx')

# 제어할 시트 지정
ws = wb.Worksheets('Sheet1')

# Q2 ~ Q1001 까지 워터토너로 채워넣는다
ws.Range("Q2").Value = "워터토너
ws.Range("Q2").AutoFill(ws.Range('Q2:Q1001'))

ws.Range("Q1002").Value = "크림토너"
ws.Range("Q1002").AutoFill(ws.Range('Q1002:Q1693'))

ws.Range("Q1694").Value = "에센스토너"
ws.Range("Q1694").AutoFill(ws.Range('Q1694:Q2693'))

ws.Range("Q2694").Value = "토너패드"
ws.Range("Q2694").AutoFill(ws.Range('Q2694:Q3693'))

ws.Range("R2").Value = "스킨/토너"
ws.Range("R2").AutoFill(ws.Range('R2:R3693'))

# 다른 제품들도 여기에 맞춰 채워넣으면 완료

In [28]:
wb.Save()

In [29]:
excel.Quit()